In [1]:
import numpy as np
import sys
import ipdb

sys.path.append("../")

sys.path.append("../utils/")
from utils_functions import *

from dd_game import *
from two_stage_player import *

# Configurations

In [2]:
#TODO: There are many configuration parameters here. It is important to understand the effect of all.
d = 2
coeff_var = 1

Sigma_x_p1 = np.eye(d)
Sigma_x_p2 = np.eye(d)

sigma_y_p1 = 1
sigma_y_p2 = 1

beta_p1 = np.random.normal(size = d)
beta_p2 = np.random.normal(size = d)

"""
mu_p1 = np.random.normal(scale = coeff_var, size = d)
mu_p2 = np.random.normal(scale = coeff_var, size = d)
gamma_p1 = np.random.normal(scale = coeff_var, size = d)
gamma_p2 = np.random.normal(scale = coeff_var, size = d)
"""

#Fixing mu and gamma for testing purposes
mu_p1 = np.array([2.1, -1.8])
gamma_p1 = np.array([1.2, 0.9])
mu_p2 = np.array([-2.8, 2.2])
gamma_p2 = np.array([0.7, -0.3])

In [3]:
p1_data_params = (Sigma_x_p1, sigma_y_p1, beta_p1, mu_p1, gamma_p1)
p1_data_generating_func = sample_from_location_family

p2_data_params = (Sigma_x_p2, sigma_y_p2, beta_p2, mu_p2, gamma_p2)
p2_data_generating_func = sample_from_location_family

num_rounds_lst = [10, 100, 1000, 10000, 40000]
num_alternate_rounds = 1000
num_test = 10000

# Run Game

In [4]:
#Initialize game with configurations specified above


for num_rounds in num_rounds_lst:
    player_two = TwoStagePlayer()
    player_one = TwoStagePlayer()
    game = DecisionDependentGame(player_one, player_two, p1_data_params, p2_data_params, 
                             p1_data_generating_func, p2_data_generating_func, 
                             num_rounds, num_alternate_rounds, num_test)
    print(game.solve_nash())
    game.run_train()
    
    print(game.evaluate_test_perf_risk_p1())
    print(game.evaluate_test_perf_risk_p2())
    
    game.run_post_train_alternating()
    
    print(game.evaluate_test_perf_risk_p1())
    print(game.evaluate_test_perf_risk_p2())
    
    #Change this flag if one wants to change print settings
    if False:
        print("---------------------")
        print("Player 1")
        print(mu_p1, gamma_p1)
        print(player_one.mu_hat, player_one.gamma_hat)

        print("Player 2")
        print(mu_p2, gamma_p2)
        print(player_two.mu_hat, player_two.gamma_hat)

(array([1.5, 0.5]), array([-4.,  4.]))
Stage 1 finished. Performing estimation now
Stage 1 finished. Performing estimation now
Stage 2 finished. Finding qs now.
Stage 2 finished. Finding qs now.
5.49075744509879
6.496634005476843
4.540195258411655
1.6003429861552052
(array([1.5, 0.5]), array([-4.,  4.]))
Stage 1 finished. Performing estimation now
Stage 1 finished. Performing estimation now
Stage 2 finished. Finding qs now.
Stage 2 finished. Finding qs now.
6.7615063995473745
10.464569907935047
2.54727293100174
1.211914533609614
(array([1.5, 0.5]), array([-4.,  4.]))
Stage 1 finished. Performing estimation now
Stage 1 finished. Performing estimation now
Stage 2 finished. Finding qs now.
Stage 2 finished. Finding qs now.
28.92096972756537
73.02078648208133
2.2001769860509635
1.0443344192530826
(array([1.5, 0.5]), array([-4.,  4.]))
Stage 1 finished. Performing estimation now
Stage 1 finished. Performing estimation now
Stage 2 finished. Finding qs now.
Stage 2 finished. Finding qs now.
6

In [5]:
if False:
    print(mu_p1, gamma_p1)
    print(player_one.mu_hat, player_one.gamma_hat)

    print(mu_p2, gamma_p2)
    print(player_two.mu_hat, player_two.gamma_hat)

    print(game.evaluate_test_perf_risk_p1())
    print(game.evaluate_test_perf_risk_p2())

In [6]:
game.solve_nash()

(array([1.5, 0.5]), array([-4.,  4.]))

## Checking stage 1

In [7]:
print(player_one.mu_hat, player_one.gamma_hat)
print(player_two.mu_hat, player_two.gamma_hat)

[ 2.1123807  -1.79650061] [1.18516172 0.90853582]
[-2.78799242  2.19420615] [ 0.71014845 -0.29098006]


In [8]:
print(mu_p1, gamma_p1)
print(mu_p2, gamma_p2)

[ 2.1 -1.8] [1.2 0.9]
[-2.8  2.2] [ 0.7 -0.3]


## Evaluate the performative risks before and after Stage 3

In [9]:
game.evaluate_test_perf_risk_p1()

2.250693442898109

In [10]:
game.evaluate_test_perf_risk_p2()

1.041499970660623

In [11]:
player_one.theta_history[-1]
player_two.theta_history[-1]

array([-0.58991312, -0.75757562])